In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('house_price_full.csv')
data.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [11]:
X = data.copy()
y = X.pop('price')

In [22]:
#Standardise the features
scaler = StandardScaler()

X = scaler.fit_transform(X)

# perform log transform of target variable
Y = np.log(y)

### Fast Forward for Single Neuron

In [23]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [28]:
#Taking 1 sample
x1, x2 = df_scaled.iloc[0]

In [29]:
#create random weight and bias
#bias
b = tf.Variable([0.1], dtype=tf.float32)

#weights
w1 = tf.Variable([0.2], dtype=tf.float32)
w2 = tf.Variable([0.15], dtype=tf.float32)

2023-01-07 16:14:28.710007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
h = (w1*x1 + w2*x2) + b
z = tf.sigmoid(h)
print("The output of the first neuron is ", z)

The output of the first neuron is  tf.Tensor([0.47511354], shape=(1,), dtype=float32)


### Fowrard propogation with multiple neurons

In [44]:
#neuron1
b1 = tf.Variable([0.1], dtype=tf.float32)
w11 = tf.Variable([0.2], dtype=tf.float32)
w12 = tf.Variable([0.15], dtype=tf.float32)

#neuron2
b2 = tf.Variable([0.25], dtype=tf.float32)
w21 = tf.Variable([0.5], dtype=tf.float32)
w22 = tf.Variable([0.6], dtype=tf.float32)

In [45]:
#Forward pass
#neuron1
z1 = b1 + (w11 * x1 + w12 * x2)
h1 = tf.math.sigmoid(z1)
print("The output from the first neuron is",h1)

The output from the first neuron is tf.Tensor([0.47511354], shape=(1,), dtype=float32)


In [46]:
#neuron2
z2 = b2 + (w21 * x1 + w22 * x2)
h2 = tf.math.sigmoid(z2)
print("The output from the second neuron is", h2)

The output from the second neuron is tf.Tensor([0.39686295], shape=(1,), dtype=float32)


In [47]:
# Layer2 weights and bias
b1 = tf.Variable([0.4])
w11 = tf.Variable([0.3])
w12 = tf.Variable([0.2])

In [48]:
#Forward pass
#neuron1

z1 = b1 + (w11 * h1 + w12 * h2)
h1 = z1
print("The output from the first neuron is", h1)

The output from the first neuron is tf.Tensor([0.62190664], shape=(1,), dtype=float32)


In [50]:
y_actual = Y[0]
y_pred = h1.numpy()

L = 0.5 * (y_actual - y_pred) ** 2
print("The MSE error is ", L)

The MSE error is  [72.38514]


### Forward propogation with matrix multiplicaton

In [52]:
#Layer 1 weight and Bias
W1 = tf.Variable([[0.2, 0.15],
                  [0.5, 0.6]])

B1 = tf.Variable([[0.1],
                  [0.25]])

In [53]:
# Layer 2 Weights and Bias
W2 = tf.Variable([[0.3, 0.2]])

B2 = tf.Variable([[0.4]])

In [55]:
X = tf.constant([[x1, x2]], dtype=tf.float32)

In [56]:
Z1 = tf.add(tf.matmul(W1, tf.transpose(X)), B1)
H1 = tf.math.sigmoid(Z1)
print(H1)

tf.Tensor(
[[0.47511354]
 [0.39686295]], shape=(2, 1), dtype=float32)


In [57]:
Z2 = tf.matmul(W2, H1) + B2
print(Z2)

tf.Tensor([[0.62190664]], shape=(1, 1), dtype=float32)


In [62]:
y_pred2 = Z2.numpy()
loss = 0.5 * (y_actual - y_pred2) ** 2
print(loss)

[[72.38514]]


### Random Weight Initilization

In [73]:
def init_random_weight():
    w1 = tf.Variable(tf.random.uniform(shape=(2, 2)))
    b1 = tf.Variable(tf.random.uniform(shape=(2, 1)))
    w2 = tf.Variable(tf.random.uniform(shape=(1, 2)))
    b2 = tf.Variable(tf.random.uniform(shape=(1, 1)))   
    
    return w1, b1, w2, b2

In [74]:
w1, b1, w2, b2 = init_random_weight()

In [66]:
print("the initial 1st layer weights is ", w1.numpy())
print("----------------------------------")
print("the initial 1st layer bias is ",    b1.numpy())
print("----------------------------------")
print("the initial 2nd layer weights is ", w2.numpy())
print("----------------------------------")
print("the initial 2nd layer bias is ",    b2.numpy())
print("----------------------------------")

the initial 1st layer weights is  [[0.97680914 0.9927825 ]
 [0.8293736  0.4609152 ]]
----------------------------------
the initial 1st layer bias is  [[0.40270472]
 [0.11293721]]
----------------------------------
the initial 2nd layer weights is  [[0.9935589  0.07338095]]
----------------------------------
the initial 2nd layer bias is  [[0.32278013]]
----------------------------------


In [107]:
def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(w1, tf.transpose(x)) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(w2, z1) + b2
    h2 = z2
    
    return h2 

In [108]:
x = tf.constant([[x1, x2]], dtype=tf.float32)
y = Y[0]

In [109]:
y_pred3 = forward_prop(x, w1, b1, w2, b2)

In [110]:
loss = 0.5 * (y_actual - y_pred3) ** 2
print("The MSE error is ", loss.numpy())

The MSE error is  [[71.336624]]


## Backpropagation

Find the value of x that minimises $y = x^2+4x$

Gradient descent update equation

$x_{new} := x_{old}-\eta\frac{dy}{dx}$

### Autogradients

In [83]:
x = tf.Variable([0.0], dtype=tf.float32)
lr = 0.1

In [85]:
#example: how to use dx/dy using tensorflow

with tf.GradientTape() as tape:
    y = x ** 2 + 4 * x

grad = tape.gradient(y, x)

In [86]:
grad.numpy()

array([4.], dtype=float32)

In [87]:
x.assign_sub(lr * grad)

<tf.Variable 'UnreadVariable' shape=(1,) dtype=float32, numpy=array([-0.4], dtype=float32)>

In [88]:
x.numpy()

array([-0.4], dtype=float32)

In [91]:
x = tf.Variable([0.0], dtype=tf.float32)
lr = 0.1
for i in range(10):
    with tf.GradientTape() as tape:
        y = x ** 2 + 4 * x

    grad = tape.gradient(y, x)
    x.assign_sub(lr * grad)
    print(x.numpy())

[-0.4]
[-0.72]
[-0.9760001]
[-1.1808001]
[-1.34464]
[-1.4757121]
[-1.5805696]
[-1.6644557]
[-1.7315645]
[-1.7852516]


### Backpropagation for Neural Networks

In [111]:
x = tf.constant([[x1, x2]], dtype=tf.float32)
y = Y[0]
lr = 0.01

def init_random_weight():
    w1 = tf.Variable(tf.random.uniform(shape=(2, 2)))
    b1 = tf.Variable(tf.random.uniform(shape=(2, 1)))
    w2 = tf.Variable(tf.random.uniform(shape=(1, 2)))
    b2 = tf.Variable(tf.random.uniform(shape=(1, 1)))   
    
    return w1, b1, w2, b2

def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(w1, tf.transpose(x)) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(w2, z1) + b2
    h2 = z2
    
    return h2

In [112]:
w1, b1, w2, b2 = init_random_weight()

In [113]:
with tf.GradientTape() as tape:
    y_pred = forward_prop(x, w1, b1, w2, b2)
    loss = 0.5 * (y - y_pred) ** 2

In [114]:
gw1, gb1, gw2, gb2 = tape.gradient(loss, [w1, b1, w2, b2])

In [116]:
print("gw1: ", gw1.numpy(), end="\n")
print("gb1: ", gb1.numpy(), end="\n")
print("gw2: ", gw2.numpy(), end="\n")
print("gb2: ", gb2.numpy(), end="\n")

gw1:  [[1.2130749  2.1093323 ]
 [0.80464435 1.3991406 ]]
gb1:  [[-2.8002806]
 [-1.8574532]]
gw2:  [[-1.7143214  6.345569 ]]
gb2:  [[-12.679347]]


In [117]:
print("Before w1: ", w1.numpy(), end="\n")
print("Before b1: ", b1.numpy(), end="\n")
print("Before w2: ", w2.numpy(), end="\n")
print("Before b2: ", b2.numpy(), end="\n")

    w1.assign_sub(gw1*lr)
    b1.assign_sub(gb1*lr)
    w2.assign_sub(gw2*lr)
    b2.assign_sub(gb2*lr)

    print("After w1: ", w1.numpy(), end="\n")
    print("After b1: ", b1.numpy(), end="\n")
    print("After w2: ", w2.numpy(), end="\n")
    print("After b2: ", b2.numpy(), end="\n")

Before w1:  [[0.06603587 0.7684941 ]
 [0.3427595  0.84697306]]
Before b1:  [[0.7426864 ]
 [0.28600645]]
Before w2:  [[0.22085369 0.14649439]]
Before b2:  [[0.01806629]]
After w1:  [[0.05390512 0.7474008 ]
 [0.33471304 0.83298165]]
After b1:  [[0.7706892]
 [0.304581 ]]
After w2:  [[0.2379969 0.0830387]]
After b2:  [[0.14485976]]


In [118]:
def train_one_step(x, y, w1, b1, w2, b2):
    y_actual = y
    
    with tf.GradientTape() as tape:
        y_pred = forward_prop(x, w1, b1, w2, b2)
        
        loss = 0.5 * (y_actual - y_pred) ** 2 
        
    #Gradient Calculation
    print("**********************************")
    print("Gradients")
    print("**********************************")
    
    gw1, gb1, gw2, gb2 = tape.gradient(loss, [w1, b1, w2, b2])
    print("The gradient of 1st layer weights are ", gw1.numpy(), end="\n")
    print("--------------------------------------")
    print("The gradient of 1st layer bias are: ", gb1.numpy(), end="\n")
    print("--------------------------------------")
    print("The gradient of 2nd layer weights are: ", gw2.numpy(), end="\n")
    print("--------------------------------------")
    print("The gradient of 2nd layer bias are: ", gb2.numpy(), end="\n")
    print("--------------------------------------")
    
    #Gradient descent
    lr = 0.2
    w1.assign_sub(gw1*lr)
    b1.assign_sub(gb1*lr)
    w2.assign_sub(gw2*lr)
    b2.assign_sub(gb2*lr)
    
    print("**********************************")
    print("New Updates")
    print("**********************************")

    print("The updated 1st layer weights are : ", w1.numpy(), end="\n")
    print("--------------------------------------")
    print("The updated 1st layer bias are:: ", b1.numpy(), end="\n")
    print("--------------------------------------")
    print("The updated 2nd layer weights are: ", w2.numpy(), end="\n")
    print("--------------------------------------")
    print("The updated 2nd layer bias are:: ", b2.numpy(), end="\n")
    print("--------------------------------------")
    
    return w1, b1, w2, b2, loss

In [119]:
w1, b1, w2, b2 = init_random_weight()
w1, b1, w2, b2, loss = train_one_step(x, y, w1, b1, w2, b2)

**********************************
Gradients
**********************************
The gradient of 1st layer weights are  [[1.7566408 3.0545015]
 [1.9066216 3.3152928]]
--------------------------------------
The gradient of 1st layer bias are:  [[-4.055056]
 [-4.401274]]
--------------------------------------
The gradient of 2nd layer weights are:  [[-3.2316666  7.3701825]]
--------------------------------------
The gradient of 2nd layer bias are:  [[-12.175979]]
--------------------------------------
**********************************
New Updates
**********************************
The updated 1st layer weights are :  [[ 0.47523126 -0.19249994]
 [ 0.61551857 -0.24311453]]
--------------------------------------
The updated 1st layer bias are::  [[1.7496514]
 [1.0231057]]
--------------------------------------
The updated 2nd layer weights are:  [[ 0.9793707 -1.1125647]]
--------------------------------------
The updated 2nd layer bias are::  [[3.0435834]]
----------------------------------